# Ceneo scraper

## Struktura opinii w Ceneo
|Składowa|selektor|zmianna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|span.user-post__author-name|author|
|rekomendację|span.user-post__author-recomendation > em|recommend|
|liczbę gwiazdek|span.user-post__score-count|stars|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datetime"]|purchased_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"] |up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|
|treść opinii|div.user-post__text|content|
|listę wad|review-feature__title--negatives ~ div.review-feature__item|cons|
|listę zalet|review-feature__title--positives ~ div.review-feature__item|pros|


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
product_id = 106579657
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

In [ ]:
response = requests.get(url)
response.status_code
response.text

In [ ]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")
opinion = page.select_one("div.js_product-review")
print(opinions)

In [ ]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            return ancestor.select_one(selector)[attribute].strip()
        return ancestor.select_one(selector).text.strip()
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()


In [ ]:

single_opinion = {

"author" : extract(opinion, "span.user-post__author-name"),
"recommend" : extract(opinion, "span.user-post__author-recomendation > em"),
"stars" : extract(opinion, "span.user-post__score-count"),
"opinion_date" : extract(opinion,("span.user-post__published > time:nth-child(1)"),"datetime"),
"purchased_date" : extract(opinion,("span.user-post__published > time:nth-child(2)"),"datetime"),
"up_vote" : extract(opinion, ("button.vote-yes"), "data-total-vote"),
"down_vote" : extract(opinion, ("button.vote-no"), "data-total-vote"),
"content" : extract(opinion, ("div.user-post__text")),
"cons" : extract(opinion, ("div.review-feature__title--negatives ~ div.review-feature__item"), None, True),
"pros" : extract(opinion, ("div.review-feature__title--positives ~ div.review-feature__item"), None, True),
"opinion_id" : extract(opinion, None,"data-entry-id")

}

print(single_opinion)